<a href="https://colab.research.google.com/github/Bravebreeze/220803314/blob/main/Exercise_3_%D0%98%D0%BD%D0%B4_%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset
from transformers import pipeline
from pathlib import Path
import time
cache_path_str = "/home/jupyter/work/cache_huggingface"
cache_path = Path(cache_path_str)
from IPython.display import HTML, display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.2 MB/s eta 0:00:00


In [ ]:
xsum_dataset = load_dataset(
    "xsum", version="1.2.0", cache_dir=cache_path
)  # Note: We specify cache_dir to use predownloaded data.
xsum_sample = xsum_dataset["train"].select(range(10))
display(xsum_sample.to_pandas())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

,document,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984
5,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,34540833
6,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,20836172
7,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,35932467
8,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,40758845
9,The crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,30358490


In [ ]:
# Constructor a summarization pipeline
summarizer = pipeline(
    task="summarization",
    model="t5-small",
    min_length=20,
    max_length=40,
    truncation=True,
    device = -1,
    model_kwargs={"cache_dir": cache_path},
)

# Apply the pipeline to the batch of articles in `xsum_sample`
summarization_results = summarizer(xsum_sample["document"])
summarization_results

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . a flood alert remains in place across the'},
 {'summary_text': 'a fire alarm went off at the Holiday Inn in Hope Street on Saturday . guests were asked to leave the hotel . the two buses were parked side-by-side in'},
 {'summary_text': 'Sebastian Vettel will start third ahead of team-mate Kimi Raikkonen . stewards only handed Hamilton a reprimand after governing body said "n'},
 {'summary_text': 'the 67-year-old is accused of committing the offences between March 1972 and October 1989 . he denies all the charges, including two counts of indecency'},
 {'summary_text': 'a man receiving psychiatric treatment at the clinic threatened to shoot himself and others . the incident comes amid tension in Istanbul following several attacks in crowded areas .'},
 {'summary_text': 'Gregor Townsend gave a debut to powerhouse wing Taqele Naiyaravor

In [ ]:
# Test your answer.
# Display the generated summary side-by-side with the reference summary and original document.
import pandas as pd

display(
    pd.DataFrame.from_dict(summarization_results)
    .rename({"summary_text": "generated_summary"}, axis=1)
    .join(pd.DataFrame.from_dict(xsum_sample))[
        ["generated_summary", "summary", "document"]
    ]
)

,generated_summary,summary,document
0,the full cost of damage in Newton Stewart is s...,Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one..."
1,a fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...
2,Sebastian Vettel will start third ahead of tea...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...
3,the 67-year-old is accused of committing the o...,A former Lincolnshire Police officer carried o...,"John Edward Bates, formerly of Spalding, Linco..."
4,a man receiving psychiatric treatment at the c...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...
5,Gregor Townsend gave a debut to powerhouse win...,Defending Pro12 champions Glasgow Warriors bag...,Simone Favaro got the crucial try with the las...
6,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille..."
7,the 25-year-old was hit by a motorbike during ...,Welsh cyclist Luke Rowe says changes to the sp...,Belgian cyclist Demoitie died after a collisio...
8,gundogan will not be fit for the start of the ...,Manchester City midfielder Ilkay Gundogan says...,"Gundogan, 26, told BBC Sport he ""can see the f..."
9,the crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,The crash happened about 07:20 GMT at the junc...


In [ ]:
jpn_dataset = load_dataset(
    "Helsinki-NLP/tatoeba_mt",
    "eng-jpn_Hani",
    cache_dir=cache_path,
)
jpn_sample = (
    jpn_dataset["test"]
    .select(range(10))
    .rename_column("sourceString", "English")
    .rename_column("targetString", "Japanese")
    .remove_columns(["sourceLang", "targetlang"])
)
display(jpn_sample.to_pandas())

README.md:   0%|          | 0.00/12.1k [00:00<?, ?B/s]

tatoeba_mt.py:   0%|          | 0.00/15.5k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

The repository for Helsinki-NLP/tatoeba_mt contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Helsinki-NLP/tatoeba_mt.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


tatoeba-test.eng-jpn_Hani.tsv:   0%|          | 0.00/147k [00:00<?, ?B/s]

tatoeba-dev.eng-jpn_Hani.tsv:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1499 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/19904 [00:00<?, ? examples/s]

,English,Japanese
0,Absolutely!,絶対！
1,A campaign is underway throughout the company ...,コピー紙の節約運動が全社的に展開されている。
2,Accrued interest will be paid into your account.,生じた利息は貯金口座に入金されます。
3,A child is very sensitive to its mother's love.,子供は母親の愛情にとても敏感だ。
4,Acid rain isn't a natural phenomenon.,酸性雨は自然現象ではない。
5,A female friend of ours took a trip to a small...,我々の女の友達は先週小さな町へ旅行しました。
6,Affection sprang up between them.,二人の間に愛情が芽生えた。
7,"After consideration, the company president mad...",社長は逡巡した後に、大規模な経営戦略の転換を図った。
8,"After I had done my homework, I went to bed.",宿題を終えた後で私は寝た。
9,"After supper, I washed the dishes.",夕食後、私は皿を洗った。


In [ ]:
# TODO

# Construct a pipeline for translating Japanese to English.
translation_pipeline = pipeline(
    task="translation",
    model="Helsinki-NLP/opus-mt-en-es",
    device = -1,
    model_kwargs={"cache_dir": cache_path},
)

# Apply your pipeline on the sample of Japanese text in: jpn_sample["Japanese"]
translation_results = translation_pipeline(jpn_sample["Japanese"])

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [ ]:
# Test your answer.
# Now we can display your translations side-by-side with the ground-truth `English` column from the dataset.
translation_results_df = pd.DataFrame.from_dict(translation_results).join(
    jpn_sample.to_pandas()
)
display(translation_results_df)

,translation_text,English,Japanese
0,"¡Oh, Dios mío!",Absolutely!,絶対！
1,,A campaign is underway throughout the company ...,コピー紙の節約運動が全社的に展開されている。
2,,Accrued interest will be paid into your account.,生じた利息は貯金口座に入金されます。
3,,A child is very sensitive to its mother's love.,子供は母親の愛情にとても敏感だ。
4,,Acid rain isn't a natural phenomenon.,酸性雨は自然現象ではない。
5,,A female friend of ours took a trip to a small...,我々の女の友達は先週小さな町へ旅行しました。
6,,Affection sprang up between them.,二人の間に愛情が芽生えた。
7,,"After consideration, the company president mad...",社長は逡巡した後に、大規模な経営戦略の転換を図った。
8,,"After I had done my homework, I went to bed.",宿題を終えた後で私は寝た。
9,,"After supper, I washed the dishes.",夕食後、私は皿を洗った。


In [ ]:
few_shot_pipeline = pipeline(
    task="text-generation",
    model="EleutherAI/gpt-neo-1.3B",
    max_new_tokens=50,
    device = -1,
    model_kwargs={"cache_dir": cache_path},
)  # Use a predownloaded model
# Get the token ID for "###", which we will use as the EOS token below.  (Recall we did this in the demo notebook.)
eos_token_id = few_shot_pipeline.tokenizer.encode("###")[0]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# TODO

# Fill in this template.

prompt =\
"""<High-level instruction about the task>:

[<input_label>]: "<input text>"
[<output_label>]: "<output_text>"
###
[<input_label>]: "<input text>"
[<output_label>]: "<output_text>"
###
[<input_label>]: "<input text>"
[<output_label>]:"""


In [ ]:
results = few_shot_pipeline(prompt, do_sample=True, eos_token_id=eos_token_id)
print(results[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:21017 for open-end generation.


<High-level instruction about the task>:

[<input_label>]: "<input text>"
[<output_label>]: "<output_text>"
###
[<input_label>]: "<input text>"
[<output_label>]: "<output_text>"
###
[<input_label>]: "<input text>"
[<output_label>]: "<output_text>"
###


In [ ]:
# Load data, tokenizer, and model.

from transformers import T5Tokenizer, T5ForConditionalGeneration

xsum_dataset = load_dataset("xsum", version="1.2.0", cache_dir=cache_path)
xsum_sample = xsum_dataset["train"].select(range(10))

tokenizer = T5Tokenizer.from_pretrained("t5-small", cache_dir=cache_path)
model = T5ForConditionalGeneration.from_pretrained(
    "t5-small", cache_dir=cache_path
)

# Prepare articles for T5, which requires a "summarize: " prefix.
articles = list(map(lambda article: "summarize: " + article, xsum_sample["document"]))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def display_summaries(decoded_summaries: list) -> None:
    """Helper method to display ground-truth and generated summaries side-by-side"""
    results_df = pd.DataFrame(zip(xsum_sample["summary"], decoded_summaries))
    results_df.columns = ["Summary", "Generated"]
    display(results_df)

In [ ]:
##############################################################################
# TODO: Try editing the parameters in this section, and see how they affect the results.
#       You can also copy and edit the cell to compare results across different parameter settings.
#
# We show all parameter settings for ease-of-modification, but in practice, you would only set relevant ones.
inputs = tokenizer(
    articles, max_length=1024, return_tensors="pt", padding=True, truncation=True
)

summary_ids = model.generate(
    inputs.input_ids,
    attention_mask=inputs.attention_mask,
    do_sample=True,
    num_beams=2,
    min_length=0,
    max_length=40,
    top_k=20,
    top_p=0.5,
    temperature=0.7,
)

decoded_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
##############################################################################

display_summaries(decoded_summaries)

,Summary,Generated
0,Clean-up operations are continuing across the ...,the full cost of damage in Newton Stewart is s...
1,Two tourist buses have been destroyed by fire ...,fire alarm went off at the Holiday Inn in Hope...
2,Lewis Hamilton stormed to pole position at the...,stewards only handed reprimand after governing...
3,A former Lincolnshire Police officer carried o...,the 67-year-old is accused of committing the o...
4,An armed man who locked himself into a room at...,a man receiving treatment at the clinic threat...
5,Defending Pro12 champions Glasgow Warriors bag...,Gregor Townsend gave a debut to powerhouse win...
6,A man with links to a car that was involved in...,"Veronica Vanessa Chango-Alverez, 31, was kille..."
7,Welsh cyclist Luke Rowe says changes to the sp...,the 25-year-old was hit by a motorbike during ...
8,Manchester City midfielder Ilkay Gundogan says...,gundogan says he can see the finishing line af...
9,A jogger has been hit by an unmarked police ca...,the crash happened about 07:20 GMT at the junc...


In [ ]:
# Options for calling the tokenizer (lots to see here)
# help(tokenizer.__call__)

In [ ]:
# Options for invoking the model (lots to see here)
# help(model.generate)

In [ ]:
# Options for calling the tokenizer for decoding (not much to see here)
# help(tokenizer.batch_decode)